In [1]:
!git clone https://github.com/timothycao/agnews-classifier.git
%cd agnews-classifier

Cloning into 'agnews-classifier'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 33 (delta 15), reused 26 (delta 8), pack-reused 0 (from 0)
Receiving objects: 100% (33/33), 8.20 KiB | 2.73 MiB/s, done.
Resolving deltas: 100% (15/15), done.
/kaggle/working/agnews-classifier


In [30]:
# Model

from peft import LoraConfig
from model import create_lora_model

lora_config = LoraConfig(
    r=3,
    lora_alpha=6,
    lora_dropout=0.1,
    bias='none',
    target_modules=['query', 'value'],
    task_type='SEQ_CLS',
    use_rslora = False # regular LoRA: lora_alpha/r, rs-LoRA: lora_alpha/sqrt(r) -> normalization technique
)

model = create_lora_model(lora_config)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 704,260 || all params: 125,352,968 || trainable%: 0.5618


In [31]:
model

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): RobertaForSequenceClassification(
      (roberta): RobertaModel(
        (embeddings): RobertaEmbeddings(
          (word_embeddings): Embedding(50265, 768, padding_idx=1)
          (position_embeddings): Embedding(514, 768, padding_idx=1)
          (token_type_embeddings): Embedding(1, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): RobertaEncoder(
          (layer): ModuleList(
            (0-11): 12 x RobertaLayer(
              (attention): RobertaAttention(
                (self): RobertaSdpaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): Mod

In [32]:
# Training

from transformers import TrainingArguments
from train import main as train

training_args = TrainingArguments(
    # Core training configs
    max_steps=2000,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    optim='adamw_torch',
    learning_rate=5e-5,
    lr_scheduler_type='linear',
    
    # Logging, evaluation, and checkpointing
    output_dir='/kaggle/working/saved_models',
    logging_strategy='steps',
    logging_steps=100,
    eval_strategy='steps',
    eval_steps=100,
    save_strategy='steps',
    save_steps=100,

    # Miscellaneous
    report_to='none',
    dataloader_num_workers=4,
    gradient_checkpointing=False,
    gradient_checkpointing_kwargs={'use_reentrant':True}
)

train(model, training_args)

Starting training...


Step,Training Loss,Validation Loss,Accuracy
100,1.366400,1.326667,0.715625
200,1.096000,0.519974,0.864062
300,0.421100,0.343587,0.884375
400,0.310400,0.330486,0.895312
500,0.311800,0.331864,0.903125
600,0.306200,0.335935,0.892188
700,0.316000,0.321524,0.901563
800,0.323600,0.309361,0.903125
900,0.306400,0.314669,0.907813
1000,0.294100,0.312265,0.904687


In [33]:
# Inference

from inference import main as inference

data_path = '/kaggle/input/deep-learning-spring-2025-project-2/test_unlabelled.pkl'
checkpoint = '/kaggle/working/saved_models/checkpoint-2000'
output_dir = '/kaggle/working/saved_predictions'

inference(data_path, checkpoint, output_dir)

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Running inference...


100%|██████████| 250/250 [00:58<00:00,  4.26it/s]

Predictions saved to /kaggle/working/saved_predictions/predictions_checkpoint-2000.csv


In [34]:
import os
import pandas as pd

# Define paths
csv_path = "/kaggle/working/saved_models/checkpoint-2000/log_history.csv"
output_dir = "/kaggle/working/processed_data"
output_csv = os.path.join(output_dir, "processed_log_history.csv")

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Read the csv file
df = pd.read_csv(csv_path)

# Select desired columns and reorder the dataframe
desired_order = [
    "step",         
    "loss",         
    "eval_loss",   
    "accuracy",     
    "eval_accuracy",
    "learning_rate",
    "epoch"         
]
df = df[desired_order]

# Rename columns for uniformity
df.rename(columns={
    "step": "Step",
    "loss": "Train Loss",
    "eval_loss": "Test Loss",
    "accuracy": "Train Acc",
    "eval_accuracy": "Test Acc",
    "learning_rate": "Learning Rate",
    "epoch": "Epochs"
}, inplace=True)

# Adding loss spread and loss ratio columns
df["loss spread"] = df["Train Loss"] - df["Test Loss"]
df["loss ratio"] = df["Train Loss"] / df["Test Loss"]

# Adding acc spread and acc ratio columns
df["Acc spread"] = df["Test Acc"] - df["Train Acc"]
df["Acc ratio"] = df["Test Acc"] / df["Train Acc"]

# Export the DataFrame as a csv file
df.to_csv(output_csv, index=False)

print(f"Data exported to {output_csv}")

df

Data exported to /kaggle/working/processed_data/processed_log_history.csv


,Step,Train Loss,Test Loss,Train Acc,Test Acc,Learning Rate,Epochs,loss spread,loss ratio,Acc spread,Acc ratio
0,100,1.3664,1.326667,0.362500,0.715625,0.000048,0.013405,0.039733,1.029949,0.353125,1.974138
1,200,1.0960,0.519974,0.717857,0.864062,0.000045,0.026810,0.576026,2.107799,0.146205,1.203669
2,300,0.4211,0.343587,0.867411,0.884375,0.000043,0.040214,0.077513,1.225600,0.016964,1.019557
3,400,0.3104,0.330486,0.895982,0.895312,0.000040,0.053619,-0.020086,0.939223,-0.000670,0.999253
4,500,0.3118,0.331864,0.896429,0.903125,0.000037,0.067024,-0.020064,0.939541,0.006696,1.007470
5,600,0.3062,0.335935,0.896429,0.892188,0.000035,0.080429,-0.029735,0.911486,-0.004241,0.995269
6,700,0.3160,0.321524,0.895536,0.901563,0.000033,0.093834,-0.005524,0.982818,0.006027,1.006730
7,800,0.3236,0.309361,0.891964,0.903125,0.000030,0.107239,0.014239,1.046026,0.011161,1.012513
8,900,0.3064,0.314669,0.897768,0.907813,0.000028,0.120643,-0.008269,0.973722,0.010045,1.011188
9,1000,0.2941,0.312265,0.902232,0.904687,0.000025,0.134048,-0.018165,0.941828,0.002455,1.002721
